In [ ]:
import marimo as mo

In [ ]:
import os
from pathlib import Path
from pprint import pprint
from typing import Self
import datetime
import pandas, pyarrow
import pydantic
import plotly.express
import altair
from ortools.sat.python import cp_model

# ジョブショップスケジューリング問題

$J || C_{\max}$ と書く.

- ジョブ $J_1, \dots, J_n$
- ジョブ $J_j$ に属するオペレーション $O_{1j}, \dots, O_{m_jj}$. この順で処理される.
- 機械 $M_1, \dots, M_m$
- オペレーション $O_{ij}$ は機械 $\mu_{ij}$ で作業時間 $p_{ij}$ かけて処理する.
- オペレーションは中断できない
- 最後のオペレーションの終了時刻を最小化

In [ ]:
parent = str(Path(os.path.abspath(__file__)).parent)
data_dir = os.path.join(parent, "data")

In [ ]:
class Task(pydantic.BaseModel):
    machine: int = pydantic.Field(..., ge=0, frozen=True)
    time: int = pydantic.Field(..., ge=0, frozen=True)

In [ ]:
class Job(pydantic.BaseModel):
    tasks: list[Task] = pydantic.Field(frozen=True)

    def from_file(fname: str) -> list[Self]:
        with open(fname) as f:
            lines = f.readlines()

        n, m = map(int, lines[0].split())
        print(f"{n=}, {m=}")

        machine, proc_time = {}, {}
        for i in range(n):
            L = list(map(int, lines[i + 1].split()))
            for j in range(m):
                machine[i, j] = L[2 * j]
                proc_time[i, j] = L[2 * j + 1]

        jobs = []
        for i in range(n):
            tasks = []
            for j in range(m):
                tasks.append(Task(machine=machine[i, j], time=proc_time[i, j]))
            jobs.append(Job(tasks=tasks))

        return jobs

In [ ]:
class Model:
    def __init__(self, jobs: list[Job]):
        self.jobs = jobs
        self.model = cp_model.CpModel()
        num_machines = len(set(task.machine for job in self.jobs for task in job.tasks))
        self.machines = list(range(num_machines))
        horizon = sum(task.time for job in self.jobs for task in job.tasks)

        self.starts = [[None for task in job.tasks] for job in jobs]
        self.intervals = [[None for task in job.tasks] for job in jobs]
        machine_to_interval = {m: [] for m in self.machines}

        for id_job, job in enumerate(self.jobs):
            for id_task, task in enumerate(job.tasks):
                suffix = f"_{id_job}_{id_task}"
                start = self.model.new_int_var(0, horizon, "start" + suffix)
                interval = self.model.new_fixed_size_interval_var(start, task.time, "interval" + suffix)
                self.starts[id_job][id_task] = start
                self.intervals[id_job][id_task] = interval
                machine_to_interval[task.machine].append(interval)

        for machine in machine_to_interval:
            if len(machine_to_interval[machine]) > 0:
                self.model.add_no_overlap(machine_to_interval[machine])

        for id_job, job in enumerate(self.jobs):
            for id_task, task in enumerate(job.tasks):
                if id_task > 0:
                    curr = self.intervals[id_job][id_task]
                    prev = self.intervals[id_job][id_task - 1]
                    self.model.add(curr.start_expr() >= prev.end_expr())

        makespan = self.model.new_int_var(0, horizon, "makespan")
        self.model.add_max_equality(
            makespan,
            [self.intervals[id_job][-1].end_expr() for id_job, job in enumerate(self.jobs)],
        )
        self.model.minimize(makespan)

    def solve(self):
        self.solver = cp_model.CpSolver()
        self.solver.parameters.log_search_progress = True
        self.status = self.solver.solve(self.model)

    def to_df(self) -> pandas.DataFrame:
        today = datetime.date.today()
        l = []
        for id_job, job in enumerate(self.jobs):
            for id_task, task in enumerate(job.tasks):
                start = self.solver.value(self.intervals[id_job][id_task].start_expr())
                end = start + self.jobs[id_job].tasks[id_task].time
                l.append(
                    dict(
                        job=f"job{id_job}",
                        task=f"task{id_task}",
                        resource=f"machine{self.jobs[id_job].tasks[id_task].machine}",
                        start=today + datetime.timedelta(start),
                        end=today + datetime.timedelta(end)
                    )
                )
        df = pandas.DataFrame(l)
        df["start"] = pandas.to_datetime(df["start"])
        df["end"] = pandas.to_datetime(df["end"])
        return df

In [ ]:
def plot_plotly(df: pandas.DataFrame):
    return plotly.express.timeline(
        df,
        x_start="start",
        x_end="end",
        y="resource",
        color="job",
        opacity=0.5
    )

In [ ]:
def plot_altair(df: pandas.DataFrame):
    return altair.Chart(df).mark_bar().encode(
        x="start",
        x2="end",
        y="resource",
        color="job",
    ).properties(
        width="container",
        height=400
    )

In [ ]:
fname1 = os.path.join(data_dir, "ft06.txt")
jobs1 = Job.from_file(fname1)
pprint(jobs1)

n=6, m=6
[Job(tasks=[Task(machine=2, time=1), Task(machine=0, time=3), Task(machine=1, time=6), Task(machine=3, time=7), Task(machine=5, time=3), Task(machine=4, time=6)]),
 Job(tasks=[Task(machine=1, time=8), Task(machine=2, time=5), Task(machine=4, time=10), Task(machine=5, time=10), Task(machine=0, time=10), Task(machine=3, time=4)]),
 Job(tasks=[Task(machine=2, time=5), Task(machine=3, time=4), Task(machine=5, time=8), Task(machine=0, time=9), Task(machine=1, time=1), Task(machine=4, time=7)]),
 Job(tasks=[Task(machine=1, time=5), Task(machine=0, time=5), Task(machine=2, time=5), Task(machine=3, time=3), Task(machine=4, time=8), Task(machine=5, time=9)]),
 Job(tasks=[Task(machine=2, time=9), Task(machine=1, time=3), Task(machine=4, time=5), Task(machine=5, time=4), Task(machine=0, time=3), Task(machine=3, time=1)]),
 Job(tasks=[Task(machine=1, time=3), Task(machine=3, time=3), Task(machine=5, time=9), Task(machine=0, time=10), Task(machine=4, time=4), Task(machine=2, time=1)])]


In [ ]:
model1_cpsat = Model(jobs1)
model1_cpsat.solve()


Starting CP-SAT solver v9.12.4544
Parameters: log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x43f0af3eddfba1f7)
#Variables: 37 (#ints: 1 in objective)
  - 37 in [0,197]
#kInterval: 36
#kLinMax: 1 (#expressions: 6)
#kLinear2: 30
#kNoOverlap: 6 (#intervals: 36)

Starting presolve at 0.00s
  1.92e-05s  0.00e+00d  [DetectDominanceRelations] 
  2.57e-04s  0.00e+00d  [PresolveToFixPoint] #num_loops=7 #num_dual_strengthening=1 
  4.09e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  4.42e-06s  0.00e+00d  [DetectDuplicateColumns] 
  8.27e-06s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 146 nodes and 175 arcs.
[Symmetry] Symmetry computation done. time: 1.2674e-05 dtime: 1.547e-05
  6.61e-06s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  1.82e-04s  8.14e-07d  [Probe] 
  2.63e-06s  0.00e+00d  [MaxClique] 
  1.92e-05s  0.00e+00d  [DetectDominanceRelations] 
  7.23e-05s  0.00e+00d

#Bound   0.01s best:58    next:[53,57]    reduced_costs
#3       0.01s best:57    next:[53,56]    quick_restart_no_lp
#4       0.01s best:56    next:[53,55]    quick_restart_no_lp
#5       0.01s best:55    next:[53,54]    fixed
#Done    0.01s quick_restart_no_lp
#Done    0.01s fixed
#Model   0.01s var:31/37 constraints:64/78

Task timing                                  n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                       'default_lp':         1 [  6.21ms,   6.21ms]   6.21ms   0.00ns   6.21ms         1 [138.65us, 138.65us] 138.65us   0.00ns 138.65us
                 'feasibility_pump':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                            'fixed':         1 [  6.06ms,   6.06ms]   6.06ms   0.00ns   6.06ms         1 [117.30us, 117.30us] 117.30us   0.00ns 117.30us
                               'fj':         0 [  0.00ns,   

In [ ]:
plot_plotly(model1_cpsat.to_df())

<marimo-plotly data-figure='{"data": [{"base": ["2025-03-27T00:00:00.000000000", "2025-03-28T00:00:00.000000000", "2025-04-07T00:00:00.000000000", "2025-04-13T00:00:00.000000000", "2025-05-03T00:00:00.000000000", "2025-05-10T00:00:00.000000000"], "hovertemplate": "job=job0<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job0", "marker": {"color": "#636efa", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job0", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i4", "bdata": "AFwmBQAUcw8AKOYeAIQMJAAUcw8AKOYe"}, "xaxis": "x", "y": ["machine2", "machine0", "machine1", "machine3", "machine5", "machine4"], "yaxis": "y", "type": "bar"}, {"base": ["2025-03-22T00:00:00.000000000", "2025-03-30T00:00:00.000000000", "2025-04-04T00:00:00.000000000", "2025-04-19T00:00:00.000000000", "2025-04-29T00:00:00.000000000", "2025-05-09T00:00:00.000000000"], "hovertemplate": "job=job1<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job1", "marker": {"color": "#EF553B", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job1", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i4", "bdata": "AOAyKQDMvxkAmH8zAJh/MwCYfzMAcJkU"}, "xaxis": "x", "y": ["machine1", "machine2", "machine4", "machine5", "machine0", "machine3"], "yaxis": "y", "type": "bar"}, {"base": ["2025-03-22T00:00:00.000000000", "2025-03-27T00:00:00.000000000", "2025-03-31T00:00:00.000000000", "2025-04-09T00:00:00.000000000", "2025-04-18T00:00:00.000000000", "2025-04-21T00:00:00.000000000"], "hovertemplate": "job=job2<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job2", "marker": {"color": "#00cc96", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job2", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i4", "bdata": "AMy/GQBwmRQA4DIpADxZLgBcJgUAhAwk"}, "xaxis": "x", "y": ["machine2", "machine3", "machine5", "machine0", "machine1", "machine4"], "yaxis": "y", "type": "bar"}, {"base": ["2025-03-30T00:00:00.000000000", "2025-04-04T00:00:00.000000000", "2025-04-13T00:00:00.000000000", "2025-04-20T00:00:00.000000000", "2025-04-28T00:00:00.000000000", "2025-05-06T00:00:00.000000000"], "hovertemplate": "job=job3<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job3", "marker": {"color": "#ab63fa", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job3", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i4", "bdata": "AMy/GQDMvxkAzL8ZABRzDwDgMikAPFku"}, "xaxis": "x", "y": ["machine1", "machine0", "machine2", "machine3", "machine4", "machine5"], "yaxis": "y", "type": "bar"}, {"base": ["2025-04-04T00:00:00.000000000", "2025-04-13T00:00:00.000000000", "2025-04-16T00:00:00.000000000", "2025-04-29T00:00:00.000000000", "2025-05-09T00:00:00.000000000", "2025-05-13T00:00:00.000000000"], "hovertemplate": "job=job4<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job4", "marker": {"color": "#FFA15A", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job4", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i4", "bdata": "ADxZLgAUcw8AzL8ZAHCZFAAUcw8AXCYF"}, "xaxis": "x", "y": ["machine2", "machine1", "machine4", "machine5", "machine0", "machine3"], "yaxis": "y", "type": "bar"}, {"base": ["2025-04-04T00:00:00.000000000", "2025-04-07T00:00:00.000000000", "2025-04-10T00:00:00.000000000", "2025-04-19T00:00:00.000000000", "2025-05-06T00:00:00.000000000", "2025-05-10T00:00:00.000000000"], "hovertemplate": "job=job5<br>start=%{base}<br>end=%{x}<br>resource=%{y}<extra></extra>", "legendgroup": "job5", "marker": {"color": "#19d3f3", "opacity": 0.5, "pattern": {"shape": ""}}, "name": "job5", "orientation": "h", "showlegend": true, "textposition": "auto", "x": {"dtype": "i4", "bdata": "ABRzDwAUcw8APFkuAJh/MwBwmRQAXCYF"}, "xaxis": "x", "y": ["machine1", "machine3", "machine5", "machine0", "machine4", "machine2

In [ ]:
mo.ui.altair_chart(plot_altair(model1_cpsat.to_df()))

&lt;marimo-vega data-initial-value=&#x27;{}&#x27; data-label=&#x27;null&#x27; data-spec=&#x27;{&amp;quot;config&amp;quot;: {&amp;quot;view&amp;quot;: {&amp;quot;continuousWidth&amp;quot;: 300, &amp;quot;continuousHeight&amp;quot;: 300}}, &amp;quot;data&amp;quot;: {&amp;quot;url&amp;quot;: &amp;quot;data:application/vnd.apache.arrow.file;base64,QVJST1cxAAD/////2AQAABAAAAAAAAoADgAGAAUACAAKAAAAAAEEABAAAAAAAAoADAAAAAQACAAKAAAAsAMAAAQAAAABAAAADAAAAAgADAAEAAgACAAAAIgDAAAEAAAAeAMAAHsiaW5kZXhfY29sdW1ucyI6IFt7ImtpbmQiOiAicmFuZ2UiLCAibmFtZSI6IG51bGwsICJzdGFydCI6IDAsICJzdG9wIjogMzYsICJzdGVwIjogMX1dLCAiY29sdW1uX2luZGV4ZXMiOiBbeyJuYW1lIjogbnVsbCwgImZpZWxkX25hbWUiOiBudWxsLCAicGFuZGFzX3R5cGUiOiAidW5pY29kZSIsICJudW1weV90eXBlIjogIm9iamVjdCIsICJtZXRhZGF0YSI6IHsiZW5jb2RpbmciOiAiVVRGLTgifX1dLCAiY29sdW1ucyI6IFt7Im5hbWUiOiAiam9iIiwgImZpZWxkX25hbWUiOiAiam9iIiwgInBhbmRhc190eXBlIjogInVuaWNvZGUiLCAibnVtcHlfdHlwZSI6ICJvYmplY3QiLCAibWV0YWRhdGEiOiBudWxsfSwgeyJuYW1lIjogInRhc2siLCAiZmllbGRfbmFtZSI6ICJ0YXNrIiwgInBhbmRhc190eXBlIjogInVuaWNvZGUiLCAibnVtcHlfdHlwZSI6ICJvYmplY3QiLCAibWV0YWRhdGEiOiBudWxsfSwgeyJuYW1lIjogInJlc291cmNlIiwgImZpZWxkX25hbWUiOiAicmVzb3VyY2UiLCAicGFuZGFzX3R5cGUiOiAidW5pY29kZSIsICJudW1weV90eXBlIjogIm9iamVjdCIsICJtZXRhZGF0YSI6IG51bGx9LCB7Im5hbWUiOiAic3RhcnQiLCAiZmllbGRfbmFtZSI6ICJzdGFydCIsICJwYW5kYXNfdHlwZSI6ICJkYXRldGltZSIsICJudW1weV90eXBlIjogImRhdGV0aW1lNjRbbnNdIiwgIm1ldGFkYXRhIjogbnVsbH0sIHsibmFtZSI6ICJlbmQiLCAiZmllbGRfbmFtZSI6ICJlbmQiLCAicGFuZGFzX3R5cGUiOiAiZGF0ZXRpbWUiLCAibnVtcHlfdHlwZSI6ICJkYXRldGltZTY0W25zXSIsICJtZXRhZGF0YSI6IG51bGx9XSwgImNyZWF0b3IiOiB7ImxpYnJhcnkiOiAicHlhcnJvdyIsICJ2ZXJzaW9uIjogIjE5LjAuMSJ9LCAicGFuZGFzX3ZlcnNpb24iOiAiMi4yLjMifQAAAAAGAAAAcGFuZGFzAAAFAAAA0AAAAJQAAABkAAAAMAAAAAQAAABU////AAABChAAAAAUAAAABAAAAAAAAAADAAAAZW5kANb///8AAAMAfP///wAAAQoQAAAAHAAAAAQAAAAAAAAABQAAAHN0YXJ0AAYACAAGAAYAAAAAAAMArP///wAAAQUQAAAAHAAAAAQAAAAAAAAACAAAAHJlc291cmNlAAAAAKT////Y////AAABBRAAAAAYAAAABAAAAAAAAAAEAAAAdGFzawAAAADM////EAAUAAgABgAHAAwAAAAQABAAAAAAAAEFEAAAABgAAAAEAAAAAAAAAAMAAABqb2IABAAEAAQAAAD/////eAEAABQAAAAAAAAADAAWAAYABQAIAAwADAAAAAADBAAYAAAAcAYAAAAAAAAAAAoAGAAMAAQACAAKAAAA7AAAABAAAAAkAAAAAAAAAAAAAAANAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlAAAAAAAAACYAAAAAAAAAJAAAAAAAAAAKAEAAAAAAAAAAAAAAAAAACgBAAAAAAAAlAAAAAAAAADAAQAAAAAAALQAAAAAAAAAeAIAAAAAAAAAAAAAAAAAAHgCAAAAAAAAlAAAAAAAAAAQAwAAAAAAACABAAAAAAAAMAQAAAAAAAAAAAAAAAAAADAEAAAAAAAAIAEAAAAAAABQBQAAAAAAAAAAAAAAAAAAUAUAAAAAAAAgAQAAAAAAAAAAAAAFAAAAJAAAAAAAAAAAAAAAAAAAACQAAAAAAAAAAAAAAAAAAAAkAAAAAAAAAAAAAAAAAAAAJAAAAAAAAAAAAAAAAAAAACQAAAAAAAAAAAAAAAAAAAAAAAAABAAAAAgAAAAMAAAAEAAAABQAAAAYAAAAHAAAACAAAAAkAAAAKAAAACwAAAAwAAAANAAAADgAAAA8AAAAQAAAAEQAAABIAAAATAAAAFAAAABUAAAAWAAAAFwAAABgAAAAZAAAAGgAAABsAAAAcAAAAHQAAAB4AAAAfAAAAIAAAACEAAAAiAAAAIwAAACQAAAAAAAAAGpvYjBqb2Iwam9iMGpvYjBqb2Iwam9iMGpvYjFqb2Ixam9iMWpvYjFqb2Ixam9iMWpvYjJqb2Iyam9iMmpvYjJqb2Iyam9iMmpvYjNqb2Izam9iM2pvYjNqb2Izam9iM2pvYjRqb2I0am9iNGpvYjRqb2I0am9iNGpvYjVqb2I1am9iNWpvYjVqb2I1am9iNQAAAAAFAAAACgAAAA8AAAAUAAAAGQAAAB4AAAAjAAAAKAAAAC0AAAAyAAAANwAAADwAAABBAAAARgAAAEsAAABQAAAAVQAAAFoAAABfAAAAZAAAAGkAAABuAAAAcwAAAHgAAAB9AAAAggAAAIcAAACMAAAAkQAAAJYAAACbAAAAoAAAAKUAAACqAAAArwAAALQAAAAAAAAAdGFzazB0YXNrMXRhc2sydGFzazN0YXNrNHRhc2s1dGFzazB0YXNrMXRhc2sydGFzazN0YXNrNHRhc2s1dGFzazB0YXNrMXRhc2sydGFzazN0YXNrNHRhc2s1dGFzazB0YXNrMXRhc2sydGFzazN0YXNrNHRhc2s1dGFzazB0YXNrMXRhc2sydGFzazN0YXNrNHRhc2s1dGFzazB0YXNrMXRhc2sydGFzazN0YXNrNHRhc2s1AAAAAAAAAAAIAAAAEAAAABgAAAAgAAAAKAAAADAAAAA4AAAAQAAAAEgAAABQAAAAWAAAAGAAAABoAAAAcAAAAHgAAACAAAAAiAAAAJAAAACYAAAAoAAAAKgAAACwAAAAuAAAAMAAAADIAAAA0AAAANgAAADgAAAA6AAAAPAAAAD4AAAAAAEAAAgBAAAQAQAAGAEAACABAAAAAAAAbWFjaGluZTJtYWNoaW5lMG1hY2hpbmUxbWFjaGluZTNtYWNoaW5lNW1hY2hpbmU0bWFjaGluZTFtYWNoaW5lMm1hY2hpbmU0bWFjaGluZTVtYWNoaW5lMG1hY2hpbmUzbWFjaGluZTJtYWNoaW5lM21hY2hpbmU1bWFjaGluZTBtYWNoaW5lMW1hY2hpbmU0bWFjaGluZTFtYWNoaW5lMG1hY2hpbmUybWFjaGluZTNtYWNoaW5lNG1hY2hpbmU1bWFjaGluZTJtYWNoaW5lMW1hY2hpbmU0bWFjaGluZTVtYWNoaW5lMG1hY2hpbmUzbWFjaGluZTFtYWNoaW5lM21hY2hpbmU1bWFjaGluZTBtYWNoaW5lNG1hY2hpbmUyAACS/9J/MBgAAOGQZ84wGAAA9z014DMYAADRpbC3NRgAAP3/S9s7GAAAJvlbAT4YAAAHKez2LhgAAH+zkGsxGAAACop39DIYAACrDSyPNxgAAMG6+aA6GAAA12fHsj0YAAAHKez2LhgAAJL/0n8wGAAAzkQlujEYAACVYF59NBgAAFx8l0A3GAAASTBVLDgYAAB/s5BrMRgAAAqKd/QyGAAA0aWwtzUYAAD6nsDdNxgAAHIpZVI6GAAA6rMJxzwYAAAKinf0MhgAANGlsLc1GAAAvlluozYYAADBuvmgOhgAANdnx7I9GAAAE60Z7T4YAAAKinf0MhgAAPc9NeAzGAAA5PHyyzQYAACrDSyPNxgAAOqzCcc8GAAAJvlbAT4YAADhkGfOMBgAAM5EJboxGAAA0aWwtzUYAAD6nsDdNxgAAOqzCcc8GAAAAGHX2D8YAAB/s5BrMRgAAAqKd/QyGAAAIDdFBjYYAADBuvmgOhgAANdnx7I9GAAAE60Z7T4YAACS/9J/MBgAAM5EJboxGAAARs/JLjQYAABcfJdANxgAAKsNLI83GAAAcillUjoYAAAKinf0MhgAAJVgXn00GAAAXHyXQDcYAADnUn7JOBgAAOqzCcc8GAAAsc9Cij8YAADRpbC3NRgAAL5ZbqM2GAAASTBVLDgYAAD9/0vbOxgAAMQbhZ4+GAAAYj6uOz8YAAD3PTXgMxgAAOTx8ss0GAAAqw0sjzcYAADBuvmgOhgAACb5WwE+GAAAdYrwTz4Y/////wAAAAAQAAAADAAUAAYACAAMABAADAAAAAAABAA4AAAAJAAAAAQAAAABAAAA6AQAAAAAAACAAQAAAAAAAHAGAAAAAAAAAAAAAAAACgAMAAAABAAIAAoAAACwAwAABAAAAAEAAAAMAAAACAAMAAQACAAIAAAAiAMAAAQAAAB4AwAAeyJpbmRleF9jb2x1bW5zIjogW3sia2luZCI6ICJyYW5nZSIsICJuYW1lIjogbnVsbCwgInN0YXJ0IjogMCwgInN0b3AiOiAzNiwgInN0ZXAiOiAxfV0sICJjb2x1bW5faW5kZXhlcyI6IFt7Im5hbWUiOiBudWxsLCAiZmllbGRfbmFtZSI6IG51bGwsICJwYW5kYXNfdHlwZSI6ICJ1bmljb2RlIiwgIm51bXB5X3R5cGUiOiAib2JqZWN0IiwgIm1ldGFkYXRhIjogeyJlbmNvZGluZyI6ICJVVEYtOCJ9fV0sICJjb2x1bW5zIjogW3sibmFtZSI6ICJqb2IiLCAiZmllbGRfbmFtZSI6ICJqb2IiLCAicGFuZGFzX3R5cGUiOiAidW5pY29kZSIsICJudW1weV90eXBlIjogIm9iamVjdCIsICJtZXRhZGF0YSI6IG51bGx9LCB7Im5hbWUiOiAidGFzayIsICJmaWVsZF9uYW1lIjogInRhc2siLCAicGFuZGFzX3R5cGUiOiAidW5pY29kZSIsICJudW1weV90eXBlIjogIm9iamVjdCIsICJtZXRhZGF0YSI6IG51bGx9LCB7Im5hbWUiOiAicmVzb3VyY2UiLCAiZmllbGRfbmFtZSI6ICJyZXNvdXJjZSIsICJwYW5kYXNfdHlwZSI6ICJ1bmljb2RlIiwgIm51bXB5X3R5cGUiOiAib2JqZWN0IiwgIm1ldGFkYXRhIjogbnVsbH0sIHsibmFtZSI6ICJzdGFydCIsICJmaWVsZF9uYW1lIjogInN0YXJ0IiwgInBhbmRhc190eXBlIjogImRhdGV0aW1lIiwgIm51bXB5X3R5cGUiOiAiZGF0ZXRpbWU2NFtuc10iLCAibWV0YWRhdGEiOiBudWxsfSwgeyJuYW1lIjogImVuZCIsICJmaWVsZF9uYW1lIjogImVuZCIsICJwYW5kYXNfdHlwZSI6ICJkYXRldGltZSIsICJudW1weV90eXBlIjogImRhdGV0aW1lNjRbbnNdIiwgIm1ldGFkYXRhIjogbnVsbH1dLCAiY3JlYXRvciI6IHsibGlicmFyeSI6ICJweWFycm93IiwgInZlcnNpb24iOiAiMTkuMC4xIn0sICJwYW5kYXNfdmVyc2lvbiI6ICIyLjIuMyJ9AAAAAAYAAABwYW5kYXMAAAUAAADQAAAAlAAAAGQAAAAwAAAABAAAAFT///8AAAEKEAAAABQAAAAEAAAAAAAAAAMAAABlbmQA1v///wAAAwB8////AAABChAAAAAcAAAABAAAAAAAAAAFAAAAc3RhcnQABgAIAAYABgAAAAAAAwCs////AAABBRAAAAAcAAAABAAAAAAAAAAIAAAAcmVzb3VyY2UAAAAApP///9j///8AAAEFEAAAABgAAAAEAAAAAAAAAAQAAAB0YXNrAAAAAMz///8QABQACAAGAAcADAAAABAAEAAAAAAAAQUQAAAAGAAAAAQAAAAAAAAAAwAAAGpvYgAEAAQABAAAAAAFAABBUlJPVzE=&amp;quot;, &amp;quot;format&amp;quot;: {&amp;quot;type&amp;quot;: &amp;quot;arrow&amp;quot;}}, &amp;quot;mark&amp;quot;: {&amp;quot;type&amp;quot;: &amp;quot;bar&amp;quot;}, &amp;quot;encoding&amp;quot;: {&amp;quot;color&amp;quot;: {&amp;quot;field&amp;quot;: &amp;quot;job&amp;quot;, &amp;quot;type&amp;quot;: &amp;quot;nominal&amp;quot;}, &amp;quot;x&amp;quot;: {&amp;quot;field&amp;quot;: &amp;quot;start&amp;quot;, &amp;quot;type&amp;quot;: &amp;quot;temporal&amp;quot;}, &amp;quot;x2&amp;quot;: {&amp;quot;field&amp;quot;: &amp;quot;end&amp;quot;}, &amp;quot;y&amp;quot;: {&amp;quot;field&amp;quot;: &amp;quot;resource&amp;quot;, &amp;quot;type&amp;quot;: &amp;quot;nominal&amp;quot;}}, &amp;quot;height&amp;quot;: 400, &amp;quot;width&amp;quot;: &amp;quot;container&amp;quot;, &amp;quot;&amp;#36;schema&amp;quot;: &amp;quot;https://vega.github.io/schema/vega-lite/v5.20.1.json&amp;quot;}&#x27; data-chart-selection=&#x27;true&#x27; data-field-selection=&#x27;true&#x27;&gt;&lt;/marimo-vega&gt;

In [ ]:
plot_altair(model1_cpsat.to_df())